**仅使用合成数据训练模型，在真实数据上测试**

# 数据划分

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder,StandardScaler
import random

In [2]:
import torch
import torch.nn as nn
import os
import random
import numpy as np

def set_seed(seed=42):
    seed = int(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv('./Data/insurance_claims_predeal.csv')
data.shape

(1000, 33)

In [4]:
Counter(data.label)

Counter({1: 247, 0: 753})

**数据集划分2**

测试集：正样本47 + 负样本148，占比0.241  
剩余：正样本200 + 负样本605 【五折交叉验证】   
* 验证集：正样本40 + 负样本121 ,占比0.248
* 训练集：正样本160 + 负样本484，占比0.248

In [5]:
set_seed(seed=42)
fraud_index = data[data.label == 1].index.tolist()
random.shuffle(fraud_index)
nofraud_index = data[data.label == 0].index.tolist()
random.shuffle(nofraud_index)

testmask = fraud_index[:47] + nofraud_index[:148]
random.shuffle(testmask)
data_test = pd.DataFrame.copy(data.iloc[testmask],deep=True)
data_test.index = range(data_test.shape[0])

rest_mask = fraud_index[47:]+nofraud_index[148:]
random.shuffle(rest_mask)
data_rest = pd.DataFrame.copy(data.iloc[rest_mask],deep=True)
data_rest.index = range(data_rest.shape[0])

In [6]:
def evaluate(clf,X_train,Y_train,X_test,Y_test):
    from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
    evaluate_dict = {}
    def GM(y_true,y_pred):
        import numpy as np
        recall = (y_pred[y_true==1]==1).mean()
        tnr = (y_pred[y_true==0]==0).mean()
        return np.sqrt(recall*tnr)
    
    clf.fit(X_train,Y_train)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:,1]
    
    #evaluate_dict['Accuracy'] = accuracy_score(Y_test,y_pred)
    evaluate_dict['AUC'] = roc_auc_score(Y_test,y_proba)
    evaluate_dict['Recall'] = recall_score(Y_test,y_pred,average='binary',pos_label=1)
    evaluate_dict['F1'] = f1_score(Y_test,y_pred,average='binary',pos_label=1)
    evaluate_dict['GM'] = GM(Y_test,y_pred)
    #evaluate_dict['FPR'] = (y_pred[Y_test==0]==1).mean()
    #evaluate_dict['TNR'] = (y_pred[Y_test==0]==0).mean()
    
    return evaluate_dict

In [7]:
data_rest[:3]

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_hobbies,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,label
0,254,45,0,1,0,1083.64,0,1,3,13,...,0,0,79680,13280,13280,53120,2,0,2004,0
1,289,45,2,1,2,1221.41,0,0,5,12,...,1,1,2700,300,300,2100,6,6,2006,0
2,172,35,0,0,2,1219.04,0,1,4,13,...,0,1,79750,14500,14500,50750,9,29,1999,0


# 网格调参

In [18]:
from imblearn.over_sampling import ADASYN,SMOTE
from xgboost.sklearn import XGBClassifier
from sdv.tabular import CTGAN
from sdv.sampling import Condition

In [8]:
def train_valid(data_rest,clf,osmethod,Normlization=True):
    metrics = []
    rest_nofraud_mask = data_rest[data_rest.label == 0].index.tolist()
    rest_fraud_mask = data_rest[data_rest.label == 1].index.tolist()
    for k in range(5):
        validmask = rest_fraud_mask[k*40:(k+1)*40] + rest_nofraud_mask[k*121:(k+1)*121]
        trainmask = list(set(rest_fraud_mask+rest_nofraud_mask)-set(validmask))
        data_train = pd.DataFrame.copy(data_rest.iloc[trainmask],deep=True)
        data_valid = pd.DataFrame.copy(data_rest.iloc[validmask],deep=True)
        data_train.index = range(data_train.shape[0])
        data_valid.index = range(data_valid.shape[0])
        if osmethod =='CTGAN':
            data_majority = data_train[data_train['label']==0]
            data_minority = data_train[data_train['label']==1]
            num_rows = data_majority.shape[0]

            ctgan = CTGAN()
            ctgan.fit(data_majority)
            condition = Condition({ 'label': 0}, num_rows)
            df_majority = ctgan.sample_conditions(conditions=[condition])

            ctgan = CTGAN()
            ctgan.fit(data_minority)
            condition = Condition({ 'label': 1}, num_rows)
            df_minority = ctgan.sample_conditions(conditions=[condition])

            new_data_train = pd.concat([df_majority,df_minority],axis=0)
            new_Xtrain, new_Ytrain = new_data_train.values[:,:-1], new_data_train.values[:,-1]
        
        index = np.arange(new_Xtrain.shape[0])
        random.seed(42)
        random.shuffle(index)
        new_Xtrain = new_Xtrain[index]
        new_Ytrain = new_Ytrain[index]

        X_valid = data_valid.values[:,:-1]
        Y_valid = data_valid.values[:,-1]

        if Normlization:
            new_Xtrain = StandardScaler().fit_transform(new_Xtrain)
            X_valid = StandardScaler().fit_transform(X_valid)

        evaluate_dict = evaluate(clf,new_Xtrain,new_Ytrain,X_valid,Y_valid)
        metrics.append(list(evaluate_dict.values()))
    return metrics

In [9]:
best_score = 0
for n_estimators in [150]:
    for max_depth in [5]:
        for learning_rate in [0.01]:
            clf=XGBClassifier(n_estimators=n_estimators,
                              max_depth = max_depth,
                              learning_rate = learning_rate
                              ) 
            metrics = train_valid(data_rest, clf, osmethod ='CTGAN', Normlization=True)
            data_metrics = pd.DataFrame(metrics,columns=['AUC','Recall','F1','GM'])
            score = data_metrics.AUC.mean()
            if score > best_score:
                best_score = score
                best_parameters={'n_estimators':n_estimators,
                                 'max_depth':max_depth,
                                 'learning_rate':learning_rate}

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
Sampling conditions: 100%|██████████████████████████████████████████| 484/484 [00:00<00:00, 1537.94it/s]
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init p

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A

[23:31:12] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A

Sampling conditions: 100%|██████████████████████████████████████████| 484/484 [00:00<00:00, 2068.95it/s]
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/home/xgwang/anaconda3/envs/DL39/lib/pyth

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A

[23:33:50] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [10]:
best_parameters

{'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.01}

# 测试

In [13]:
from imblearn.over_sampling import ADASYN,SMOTE
from sdv.tabular import CTGAN
from sdv.sampling import Condition
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBClassifier

In [14]:
import torch
import torch.nn as nn
import os
import random
import numpy as np

def set_seed(seed=42):
    seed = int(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True

In [15]:
def test(data_rest,data_test,clf,osmethod='CTGAN',Normlization=True):    
    random.seed(42)
    if osmethod =='CTGAN':
        data_majority = data_rest[data_rest['label']==0]
        data_minority = data_rest[data_rest['label']==1]
        num_rows = data_majority.shape[0]

        ctgan = CTGAN()
        ctgan.fit(data_majority)
        condition = Condition({ 'label': 0}, num_rows)
        df_majority = ctgan.sample_conditions(conditions=[condition])

        ctgan = CTGAN()
        ctgan.fit(data_minority)
        condition = Condition({ 'label': 1}, num_rows)
        df_minority = ctgan.sample_conditions(conditions=[condition])

        new_data_train = pd.concat([df_majority,df_minority],axis=0)
        new_Xtrain, new_Ytrain = new_data_train.values[:,:-1], new_data_train.values[:,-1]
    
    index = np.arange(new_Xtrain.shape[0])
    random.seed(42)
    random.shuffle(index)
    new_Xtrain = new_Xtrain[index]
    new_Ytrain = new_Ytrain[index]
    
    X_test = data_test.values[:,:-1]
    Y_test = data_test.values[:,-1]
    
    if Normlization:
        new_Xtrain = StandardScaler().fit_transform(new_Xtrain)
        X_test = StandardScaler().fit_transform(X_test)
    
    evaluate_dict = evaluate(clf,new_Xtrain,new_Ytrain,X_test,Y_test)
    return evaluate_dict

In [16]:
set_seed(seed=42)
clf = XGBClassifier(n_estimators=150, max_depth = 5, learning_rate=0.01, random_state=42)

evaluate_dict = test(data_rest,data_test,clf,osmethod='CTGAN',Normlization=False)

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
Sampling conditions: 100%|████████████████████████████████████████████| 605/605 [00:00<00:00, 2230.41it/s]
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_

/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].to_numpy().flatten()
/home/xgwang/anaconda3/envs/DL39/lib/python3.9/site-packages/ctgan/data_transformer.py:111: SettingWithCopyWarning: 
A

[11:30:52] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [17]:
evaluate_dict

{'AUC': 0.6736630247268546,
 'Recall': 0.574468085106383,
 'F1': 0.45762711864406785,
 'GM': 0.6353583839225714}